In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_0

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
from os.path import join
import datetime
import dateutil.parser

from constants_1_0 import COLUMNS, DATA_AGGREGATE_TYPES
from utils_1_0 import (
    read_combined_daily_counts_df, 
    read_combined_by_country_daily_counts_df,
    apply_theme, get_country_color_map,
    get_visualization_subtitle
)
from web import for_website

In [ ]:
CATEGORY = "category"
CATEGORY_OF_INTEREST = "new_positive_cases"

country_color_map = get_country_color_map()

min_date = datetime.datetime(2020, 1, 27) + datetime.timedelta(hours=1)
max_date = datetime.datetime(2020, 4, 2) + datetime.timedelta(hours=1)

# Countries have different ids in the JHU data than in the 4CE data
country_map = {
    "US": "USA"
}

In [ ]:
def preprocess_daily_df(df_dc):
    # Adapted from 02_daily_counts_altair.ipynb
    CATEGORY = "category"

    # Wide to long
    df_dc = pd.melt(df_dc, id_vars=[
        COLUMNS.SITE_ID, COLUMNS.DATE,
        COLUMNS.MASKED_UPPER_BOUND_NEW_POSITIVE_CASES,
        COLUMNS.MASKED_UPPER_BOUND_PATIENTS_IN_ICU,
        COLUMNS.MASKED_UPPER_BOUND_NEW_DEATHS,
        COLUMNS.UNMASKED_SITES_NEW_POSITIVE_CASES,
        COLUMNS.UNMASKED_SITES_PATIENTS_IN_ICU,
        COLUMNS.UNMASKED_SITES_NEW_DEATHS,
        COLUMNS.MASKED_SITES_NEW_POSITIVE_CASES,
        COLUMNS.MASKED_SITES_PATIENTS_IN_ICU,
        COLUMNS.MASKED_SITES_NEW_DEATHS
    ])
    df_dc = df_dc.rename(columns={"variable": CATEGORY, "value": COLUMNS.NUM_PATIENTS})

    # Leave only the 'upper' and 'under' values for the certain 'category' only
    for c in [COLUMNS.NEW_POSITIVE_CASES, COLUMNS.PATIENTS_IN_ICU, COLUMNS.NEW_DEATHS]:
        filter_c = df_dc[CATEGORY] == c
        df_dc.loc[filter_c, "upper"] = df_dc.loc[filter_c, COLUMNS.NUM_PATIENTS] + df_dc.loc[filter_c, "masked_upper_bound_" + c]
        df_dc.loc[filter_c, "under"] = df_dc.loc[filter_c, COLUMNS.NUM_PATIENTS]
        df_dc.loc[filter_c, COLUMNS.NUM_PATIENTS] = df_dc.loc[filter_c, COLUMNS.NUM_PATIENTS] + df_dc.loc[filter_c, "masked_upper_bound_" + c] / 2.0
        
        # Add num of sites
        df_dc.loc[filter_c, COLUMNS.NUM_SITES] = df_dc["unmasked_sites_" + c] + df_dc["masked_sites_" + c]

    # Drop unused columns
    df_dc = df_dc.drop(columns=[
        COLUMNS.MASKED_UPPER_BOUND_NEW_POSITIVE_CASES,
        COLUMNS.MASKED_UPPER_BOUND_PATIENTS_IN_ICU,
        COLUMNS.MASKED_UPPER_BOUND_NEW_DEATHS,
        COLUMNS.UNMASKED_SITES_NEW_POSITIVE_CASES,
        COLUMNS.UNMASKED_SITES_PATIENTS_IN_ICU,
        COLUMNS.UNMASKED_SITES_NEW_DEATHS,
        COLUMNS.MASKED_SITES_NEW_POSITIVE_CASES,
        COLUMNS.MASKED_SITES_PATIENTS_IN_ICU,
        COLUMNS.MASKED_SITES_NEW_DEATHS
    ])
    
    # Remove zero num_sites
    df_dc = df_dc[df_dc[COLUMNS.NUM_SITES] != 0]

    df_dc = df_dc.loc[df_dc["category"] == CATEGORY_OF_INTEREST]
    df_dc = df_dc.rename(columns={"siteid": "country", "num_patients": "count"})

    return df_dc


# DailyCounts-CombinedByCountry.csv
df_dc = pd.read_csv("https://ndownloader.figshare.com/files/22346625")

df_dc = preprocess_daily_df(df_dc)
df_dc.head()

In [ ]:
# We only need the JHU data for the countries that exist in the 4CE data.
unique_countries = df_dc["country"].unique().tolist()
unique_countries

In [ ]:
# Parse date strings into date objects.
def convert_date(date_str):
    try:
        return dateutil.parser.parse(date_str)
    except:
        return np.nan

In [ ]:
# Transform the JHU data.
jhu_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/dcd4181613f512a6f75249fc77b63286aebe7271/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
jhu_df = pd.read_csv(jhu_url)

jhu_df = jhu_df.rename(columns={"Country/Region": "country", "Province/State": "state"})
jhu_df = jhu_df.drop(columns=["Lat", "Long"])

jhu_df["country"] = jhu_df["country"].apply(lambda c: country_map[c] if c in country_map else c)
jhu_df = jhu_df.loc[jhu_df["country"].isin(unique_countries)]
jhu_df = jhu_df.loc[~pd.notna(jhu_df["state"])]
jhu_df = jhu_df.drop(columns=["state"])

jhu_df = jhu_df.melt(id_vars=["country"], var_name="date", value_name="cumulative_count")
jhu_df["date"] = jhu_df["date"].astype(str)
jhu_df["date"] = jhu_df["date"].apply(convert_date)
jhu_df = jhu_df.sort_values(by="date", ascending=True)
jhu_df = jhu_df.loc[(jhu_df["date"] >= min_date) & (jhu_df["date"] <= max_date)]
jhu_df["date_str"] = jhu_df["date"].astype(str)

jhu_df_freeze = jhu_df.copy()

jhu_roc_df = pd.DataFrame(index=[], data=[], columns=["country", "date", "cumulative_count"])
for country, country_df in jhu_df.groupby("country"):
    country_df = country_df.copy()
    country_df["count"] = np.concatenate((np.array([np.nan]), np.diff(country_df["cumulative_count"].values)))
    country_df["cumulative_count"] = country_df["cumulative_count"].replace(0, np.nan)
    
    country_df["N0"] = country_df["cumulative_count"].shift(1) # N0 is the total case up to the day before
    country_df["n1"] = country_df["count"] # n1 is the case number this day
    country_df["n2"] = country_df["n1"].shift(1) # n2 is the case number yesterday
    
    country_df["percent_increase"] = (country_df["n1"] / country_df["N0"]) * 100
    
    country_df['R'] = country_df["percent_increase"] # TODO: is this correct?
    # TODO: update CI formula
    country_df['C'] = country_df['R'] - 1
    country_df['standard_error'] = country_df.apply(lambda obs: (obs['R']+np.power(obs['R'], 2))/obs['n2'], axis='columns')
    country_df['95_CI_below'] = country_df.apply(lambda obs: obs['C'] - 1.96*np.sqrt(obs['standard_error']), axis='columns')
    country_df['95_CI_above'] = country_df.apply(lambda obs: obs['C'] + 1.96*np.sqrt(obs['standard_error']), axis='columns')
    country_df = country_df.replace([np.inf, -np.inf], np.nan)


    jhu_roc_df = jhu_roc_df.append(country_df, ignore_index=True)
jhu_roc_df.head()

In [ ]:
# Transform the 4CE data to obtain normalized change values.
df_dc = df_dc.loc[df_dc["category"] == CATEGORY_OF_INTEREST]
df_dc["date"] = df_dc["date"].astype(str)
df_dc["date"] = df_dc["date"].apply(convert_date)
df_dc = df_dc.sort_values(by="date", ascending=True)
df_dc = df_dc.loc[(df_dc["date"] >= min_date) & (df_dc["date"] <= max_date)]
df_dc["date_str"] = df_dc["date"].astype(str)

df_dc_freeze = df_dc.copy()

dc_roc_df = pd.DataFrame(index=[], data=[], columns=["country", "date", "count"])
for country, country_df in df_dc.groupby("country"):
    country_df = country_df.copy()
    
    country_df["cumulative_count"] = np.cumsum(country_df["count"].values)
    country_df["N0"] = country_df["cumulative_count"].shift(1) # N0 is the total case up to the day before
    country_df["n1"] = country_df["count"] # n1 is the case number this day
    country_df["n2"] = country_df["n1"].shift(1) # n2 is the case number yesterday
    
    country_df["percent_increase"] = (country_df["n1"] / country_df["N0"]) * 100
    
    country_df['R'] = country_df["percent_increase"] # TODO: is this correct?
    # TODO: update CI formula
    country_df['C'] = country_df['R'] - 1
    country_df['standard_error'] = country_df.apply(lambda obs: (obs['R']+np.power(obs['R'], 2))/obs['n2'], axis='columns')
    country_df['95_CI_below'] = country_df.apply(lambda obs: obs['C'] - 1.96*np.sqrt(obs['standard_error']), axis='columns')
    country_df['95_CI_above'] = country_df.apply(lambda obs: obs['C'] + 1.96*np.sqrt(obs['standard_error']), axis='columns')
    country_df = country_df.replace([np.inf, -np.inf], np.nan)

    dc_roc_df = dc_roc_df.append(country_df, ignore_index=True)
dc_roc_df.head()

In [ ]:
def get_jhu_cumulative_count(date_str, country):
    return jhu_roc_df.loc[(jhu_roc_df["date_str"] == date_str) & (jhu_roc_df["country"] == country)].reset_index().iloc[0]["cumulative_count"]

# Start plotting after country has 100 cases
count_threshold = 100
jhu_roc_df["jhu_past_100"] = jhu_roc_df["cumulative_count"] >= count_threshold
dc_roc_df["jhu_past_100"] = dc_roc_df.apply(lambda row: get_jhu_cumulative_count(row["date_str"], row["country"]) >= count_threshold, axis='columns')


### Transform data for plots faceted by country

In [ ]:
jhu_roc_df = jhu_roc_df.copy()
dc_roc_df = dc_roc_df.copy()
jhu_roc_df["source"] = "JHU CSSE"
dc_roc_df["source"] = "4CE"

In [ ]:
dc_roc_df.head()

In [ ]:
join_df = jhu_roc_df.append(dc_roc_df, ignore_index=True)
join_df["country_source"] = join_df.apply(lambda row: row["country"] + "_" + row["source"], axis='columns')
join_df.head()

## Normalized New Daily Cases

First, obtain total hospital discharges for each country.

$\texttt{country_total} = \text{country total in-patient-discharge}$

$\texttt{country_4CE_total} = \text{total in-patient-discharge in our sites within that country}$

$F0 = \frac{\texttt{country_total}}{\texttt{country_4CE_total}}$

$F0$ is used to normalize.

- For new figure that shows daily case # per 100K, we will instead show 
    - $\texttt{RATE} = \texttt{N_case} * F1$
    - where $F1 = F0 * \frac{100K}{\texttt{country population}}$
    - then the standard error for $\texttt{RATE}$ will be $\sqrt(\texttt{RATE}*F1)$ and the confidence interval will be $\texttt{RATE} \pm 1.96*\sqrt(\texttt{RATE}*F1)$

In [ ]:
# Get daily new cases from cumulative_count
norm_jhu = jhu_roc_df.copy()

# norm_jhu = norm_jhu[norm_jhu["country"] != "Singapore"].reset_index() # Remove Singapore

norm_jhu["count"] = norm_jhu["cumulative_count"].diff()

norm_jhu.loc[norm_jhu["date"] == "2020-01-28", "count"] = np.nan # Make sure the start count is NaN

norm_jhu

In [ ]:
norm_4ce = dc_roc_df.copy()

In [ ]:
population = {
    # Adopted from https://www.worldometers.info on May 12, 2020
    "France": 65254341,
    "USA": 330743891,
    "Germany": 83748343,
    "Italy": 60473574,
    "Singapore": 5844512
}
F0 = {
    "France": 8.06,
    "USA": 47.68,
    "Germany": 153.6,
    "Italy": 70.9,
    "Singapore": 15.84
}
norm_jhu["population"] = norm_jhu["country"]
norm_jhu["population"] = norm_jhu["population"].apply(lambda x: population[x])
norm_jhu["F0"] = norm_jhu["country"]
norm_jhu["F0"] = norm_jhu["F0"].apply(lambda x: F0[x])

norm_4ce["population"] = norm_4ce["country"]
norm_4ce["population"] = norm_4ce["population"].apply(lambda x: population[x])
norm_4ce["F0"] = norm_4ce["country"]
norm_4ce["F0"] = norm_4ce["F0"].apply(lambda x: F0[x])

norm_4ce.head()

In [ ]:
norm_jhu["adjusted_count"] = norm_jhu["count"]
norm_jhu["F1"] = 100000 / norm_jhu["population"]
norm_jhu["RATE"] = norm_jhu["count"] * norm_jhu["F1"]
norm_jhu["RATE_7_day_avg"] = norm_jhu["RATE"].rolling(7).mean().shift(-3)
norm_jhu["std_error"] = norm_jhu["F1"] * norm_jhu["RATE"]
norm_jhu["std_error"] = norm_jhu["std_error"].apply(lambda x: np.sqrt(x))
norm_jhu["ci_above"] = norm_jhu["RATE_7_day_avg"] + 1.96 * norm_jhu["std_error"]
norm_jhu["ci_below"] = norm_jhu["RATE_7_day_avg"] - 1.96 * norm_jhu["std_error"]

norm_4ce["adjusted_count"] = norm_4ce["F0"] * norm_4ce["count"]
norm_4ce["F1"] = norm_4ce["F0"] * 100000 / norm_4ce["population"]
norm_4ce["RATE"] = norm_4ce["count"] * norm_4ce["F1"]
for c in ["France", "Germany", "Italy", "USA", "Singapore"]:
    c_filter = norm_4ce["country"] == c
    norm_4ce.loc[c_filter, "RATE_7_day_avg"] = norm_4ce.loc[c_filter, "RATE"].rolling(7).mean().shift(-3)
# norm_4ce["RATE_7_day_avg"] = norm_4ce["RATE"].rolling(7).mean().shift(-3)
norm_4ce["std_error"] = norm_4ce["F1"] * norm_4ce["RATE"]
norm_4ce["std_error"] = norm_4ce["std_error"].apply(lambda x: np.sqrt(x))
norm_4ce["ci_above"] = norm_4ce["RATE_7_day_avg"] + 1.96 * norm_4ce["std_error"]
norm_4ce["ci_below"] = norm_4ce["RATE_7_day_avg"] - 1.96 * norm_4ce["std_error"]

In [ ]:
norm_jhu_min_col = norm_jhu[['country', 'date', 'count', 'adjusted_count', 'RATE_7_day_avg', 'source', 'population', 'F0', 'F1', 'RATE', 'std_error', 'ci_above', 'ci_below', 'jhu_past_100']]
norm_4ce_min_col = norm_4ce[['country', 'date', 'count', 'adjusted_count', 'RATE_7_day_avg', 'source', 'population', 'F0', 'F1', 'RATE', 'std_error', 'ci_above', 'ci_below', 'jhu_past_100', 'num_sites']]

norm_4ce_min_col.head(10)
norm_jhu_min_col.head(10)

In [ ]:
norm_df = norm_jhu_min_col.append(norm_4ce_min_col)

norm_df["country_source"] = norm_df.apply(lambda row: row["country"] + "_" + row["source"], axis='columns')

norm_df.to_csv("norm_df.csv")

norm_fce_df = norm_df.loc[norm_df['source'] == '4CE'].copy()

norm_df

In [ ]:
max_date_3 = datetime.datetime(2020, 3, 30)
norm_df = norm_df.loc[(norm_df["jhu_past_100"]) & (norm_df["date"] <= max_date_3)]

In [ ]:
title = "Country-Level Positive Case Rate, Comparison to JHU CSSE Data"

# Selection
source_selection = alt.selection_multi(fields=["source"], bind="legend")

min_date = norm_df["date"].min()
max_date = norm_df["date"].max()
norm_fce_df = norm_fce_df.loc[(norm_fce_df["date"] >= min_date) & (norm_fce_df["date"] <= max_date)]

# Domains
date_domain = [alt.DateTime(year=min_date.year, month=min_date.month, date=min_date.day), alt.DateTime(year=max_date.year, month=max_date.month, date=max_date.day)]
sites_domain = [0, norm_fce_df["num_sites"].max() + 1]
patients_domain = [0, norm_fce_df["count"].max() + 1]

country_names = ["France", "Germany", "Italy", "USA", "Singapore"]
COUNTRY_COLORS = ["#0072B2", "#E69F00", "#009E73", "#D55E00", "#CC79A7"]
country_source_names = [c + "_" + "4CE" for c in country_names] + [c + "_" + "JHU CSSE" for c in country_names]
color_scale = alt.Scale(domain=country_names, range=COUNTRY_COLORS)
join_color_scale = alt.Scale(domain=country_source_names, range=COUNTRY_COLORS + ["#707070"] * len(country_names))

country_width = 170

nearest = alt.selection_single(encodings=['x', 'y'], on="mouseover", nearest=True, empty="none", clear="mouseout")
y_selection = alt.selection_interval(encodings=["y"], bind="scales")
date_brush = alt.selection(type='interval', encodings=['x'])

# Additional Visual Elements
tooltip = [
    alt.Tooltip("source", title="Data source"),
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("count", title="Daily Cases"),
    alt.Tooltip("adjusted_count", title="Adjusted Daily Cases"),
    alt.Tooltip("RATE_7_day_avg", title="Daily Case Rate, 7-day Average"),
    alt.Tooltip("date", title="Date"),
    alt.Tooltip("ci_below", title="95% CI upper bound"),
    alt.Tooltip("ci_above", title="95% CI lower bound")
]

rule = alt.Chart().mark_rule(color="red", size=0.5).encode(
    x="date:T"
).transform_filter(
    nearest
)

line = alt.Chart(norm_df).transform_filter(source_selection).mark_line(opacity=0.7).encode(
    x=alt.X("date:T", title=None, axis=alt.Axis(labelBound=True), scale=alt.Scale(padding=5)),
    y=alt.Y("RATE_7_day_avg:Q", axis=alt.Axis(title="Adjusted daily case rate, 7 day average"), scale=alt.Scale(zero=False, nice=False, padding=5)),
    strokeDash=alt.StrokeDash("source:N", scale=alt.Scale(domain=["4CE", "JHU CSSE"], range=[[0,0], [3,3]]), 
    legend=alt.Legend(title="Data Source")),
    color=alt.Color("country_source:N", scale=join_color_scale, legend=None),
    tooltip=tooltip
).properties(width=country_width, height=200)

errorband = line.transform_filter(alt.datum["source"] == "4CE").mark_errorband().encode(
    x=alt.X(f"date:T", title=None, axis=alt.Axis(labelBound=True)),
    y=alt.Y(f"sum(ci_below):Q", title=""),
    y2=alt.Y2(f"sum(ci_above):Q", title=""),
    color=alt.Color(f"country:N", scale=color_scale, legend=alt.Legend(title=None)),
    tooltip=tooltip
)

circle = (
    line.mark_circle()
        .encode(
            size=alt.condition(~nearest, alt.value(5), alt.value(30))
        )
        .add_selection(nearest)
)

num_sites_bar_bg = (
    alt.Chart(norm_fce_df)
        .mark_bar(size=2)
        .encode(
            x=alt.X("date:T", scale=alt.Scale(domain=date_domain, padding=5), title=None, axis=alt.Axis(labelBound=True)),
            y=alt.Y("num_sites:Q", axis=alt.Axis(title="# of sites"), scale=alt.Scale(domain=sites_domain)),
            color=alt.value("gray"),
            tooltip=tooltip
        )
        .properties(width=country_width, height=60) 
)

num_sites_bar = (
    num_sites_bar_bg
        .encode(
            color=alt.Color("country:N", scale=color_scale, legend=None),
        )
        .transform_filter(date_brush)
)

num_patients_bar_bg = (
    alt.Chart(norm_fce_df)
        .mark_bar(size=2)
        .encode(
            x=alt.X("date:T", scale=alt.Scale(domain=date_domain, padding=5), title=None, axis=alt.Axis(labelBound=True)),
            y=alt.Y("count:Q", axis=alt.Axis(title="# of new cases"), scale=alt.Scale(domain=patients_domain)),
            color=alt.value("gray"),
            tooltip=tooltip
        )
        .properties(width=country_width, height=60) 
)

num_patients_bar = (
    num_patients_bar_bg
        .encode(
            color=alt.Color("country:N", scale=color_scale, legend=None),
        )
        .transform_filter(date_brush)
)

top = (
    alt.layer(line, errorband, circle, rule, data=norm_df)
        .facet(
            column=alt.Column("country:N"), bounds="flush" #header=alt.Header(labels=False)
        )
        .add_selection(y_selection)
)

num_sites_bottom = (
    alt.layer(num_sites_bar_bg, num_sites_bar, rule, data=norm_fce_df)
        .facet(
            column=alt.Column("country:N", header=alt.Header(labels=False)), bounds="flush"
        )
        .add_selection(nearest)
        .add_selection(date_brush)
)

num_patients_bottom = (
    alt.layer(num_patients_bar_bg, num_patients_bar, rule, data=norm_fce_df)
        .facet(
            column=alt.Column("country:N", header=alt.Header(labels=False)), bounds="flush"
        )
        .add_selection(nearest)
        .add_selection(date_brush)
)

plot = (
    alt.vconcat(top, num_patients_bottom, num_sites_bottom, spacing=5)
        .resolve_scale(color="shared", x="independent")
        .properties(title={
                "text": title, 
                "subtitle": get_visualization_subtitle(),
                "subtitleColor": "gray",
                "dx": 60
        })
        .add_selection(source_selection)
)


plot = apply_theme(
    plot, 
    axis_label_font_size=10, 
    axis_title_font_size=12, 
    axis_title_padding=8, 
    legend_orient="bottom", 
    legend_symbol_type="stroke",
    legend_title_orient="left",
    legend_title_font_size=14,
    label_font_size=12
).configure_header(title=None, labelPadding=3, labelFontSize=13)

for_website(plot, "Daily Count", "country-level rate of positive cases")

plot